# ClipCard Simulation: Civic/ICS

**Purpose:** This notebook runs a Monte Carlo simulation to estimate the effectiveness of ClipCard parameters for civic incident command operations.

It simulates operations and checks if randomly generated metrics (like `queue_time` or `stockout`) would trigger the `kill_criteria` defined in a ClipCard. This helps quantify the risk reduction provided by the card's specific configuration.

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import json
import glob

# Load example ClipCards from the examples or data directory
try:
    with open('../../examples/test.clipcard.json', 'r') as f:
        example_card = json.load(f)
    print(f"Loaded example ClipCard: {example_card['id']}")
except FileNotFoundError:
    print("No example ClipCard found. Using synthetic data.")
    example_card = {
        "id": "SIM-CIVIC-001",
        "kill_criteria": [
            {"condition": "queue_time > 120", "action": "Activate overflow shelter"},
            {"condition": "stockout >= 1", "action": "Request emergency supplies"}
        ],
        "authority_window": {"scope_limit": "Single Division", "time_limit": "30d"}
    }

### Simulation Parameters

Configure simulation runs and metric distributions

In [ ]:
# Simulation configuration
N_SIMULATIONS = 1000  # Number of operational simulations
COHORT_SIZE = 0.20    # 20% of shelters
BASE_QUEUE_TIME = 45  # Base queue time in minutes
BASE_STOCKOUT_RATE = 0.0005  # Base stockout probability

print(f"Running {N_SIMULATIONS} simulations...")
print(f"Shelter coverage: {COHORT_SIZE*100}%")
print(f"Base queue time: {BASE_QUEUE_TIME} min")
print(f"Base stockout rate: {BASE_STOCKOUT_RATE}")

### Monte Carlo Simulation

Run simulations and track kill criteria triggers

In [ ]:
results = []

for i in range(N_SIMULATIONS):
    # Simulate metrics for a 24-hour period
    arrivals = np.random.poisson(200)  # Number of arrivals
    
    # Simulate queue time (Gamma distribution)
    queue_time = np.random.gamma(2, BASE_QUEUE_TIME/2)
    
    # Simulate stockouts (Poisson process)
    stockouts = np.random.poisson(BASE_STOCKOUT_RATE * arrivals)
    
    # Check kill criteria
    queue_trigger = queue_time > 120
    stockout_trigger = stockouts >= 1
    any_trigger = queue_trigger or stockout_trigger
    
    results.append({
        'sim_id': i,
        'arrivals': arrivals,
        'queue_time': queue_time,
        'stockouts': stockouts,
        'queue_trigger': queue_trigger,
        'stockout_trigger': stockout_trigger,
        'any_trigger': any_trigger
    })

df = pd.DataFrame(results)
print(f"\nSimulation complete!")
print(f"Queue time triggers: {df['queue_trigger'].sum()} ({df['queue_trigger'].mean()*100:.1f}%)")
print(f"Stockout triggers: {df['stockout_trigger'].sum()} ({df['stockout_trigger'].mean()*100:.1f}%)")
print(f"Any trigger: {df['any_trigger'].sum()} ({df['any_trigger'].mean()*100:.1f}%)")

### Visualization

Plot trigger distributions and risk reduction

In [ ]:
fig, axes = plt.subplots(1, 3, figsize=(15, 4))

# Queue time distribution
axes[0].hist(df['queue_time'], bins=30, alpha=0.7, edgecolor='black')
axes[0].axvline(120, color='red', linestyle='--', label='Kill threshold')
axes[0].set_xlabel('Queue Time (minutes)')
axes[0].set_ylabel('Frequency')
axes[0].set_title('Queue Time Distribution')
axes[0].legend()

# Stockout distribution
axes[1].hist(df['stockouts'], bins=30, alpha=0.7, edgecolor='black')
axes[1].axvline(1, color='red', linestyle='--', label='Kill threshold')
axes[1].set_xlabel('Stockouts per 24h')
axes[1].set_ylabel('Frequency')
axes[1].set_title('Stockout Distribution')
axes[1].legend()

# Trigger summary
trigger_counts = [
    df['queue_trigger'].sum(),
    df['stockout_trigger'].sum(),
    df['any_trigger'].sum()
]
trigger_labels = ['Queue Time', 'Stockout', 'Any Trigger']
axes[2].bar(trigger_labels, trigger_counts, alpha=0.7, edgecolor='black')
axes[2].set_ylabel('Number of Triggers')
axes[2].set_title(f'Kill Criteria Triggers (n={N_SIMULATIONS})')
axes[2].axhline(N_SIMULATIONS * 0.05, color='orange', linestyle='--', label='5% threshold')
axes[2].legend()

plt.tight_layout()
plt.show()

print(f"\n** Interpretation **")
print(f"In {N_SIMULATIONS} simulated 24h periods with a {COHORT_SIZE*100}% coverage:")
print(f"- Kill criteria would trigger {df['any_trigger'].mean()*100:.1f}% of the time")
print(f"- This suggests the authority window provides {(1-df['any_trigger'].mean())*100:.1f}% safe operation probability")
print(f"- Estimated incident prevention: {df['any_trigger'].sum()} potential incidents caught early")